## 定义使用的模型

In [1]:
from langchain_community.llms import Ollama
# 创建 Ollama 实例
# 确保 Ollama 服务正在运行，并且模型已经拉取到本地
llm = Ollama(
    base_url='http://127.0.0.1:11434',  # 本地 Ollama 服务地址 其实默认就是这个地址
    model='qwen2:7b',  # 你要使用的模型名称
    timeout=60
)

C:\Users\sky92\AppData\Local\Temp\ipykernel_8432\3643654744.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


## 调用本地ollama 保留链式处理 完整例子

In [ ]:
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
 
# 创建 PromptTemplate 模板定义
# my_prompt = PromptTemplate.from_template("""回答这个问题:{input}""")

my_prompt = PromptTemplate.from_template([
    ("system","你是世界级技术专家"),
    ("user","{input}"),
])

# 创建 Ollama 实例
# 确保 Ollama 服务正在运行，并且模型已经拉取到本地
ollama = Ollama(
    base_url='http://127.0.0.1:11434',  # 本地 Ollama 服务地址
    model='qwen2:7b',  # 你要使用的模型名称
    timeout=60
)

# 链式动作可调用api 调用数据库  爬虫 结果储存 等动作
# 创建链式调用  模板定义->大模型处理->输出内容解析   
chain = my_prompt | ollama | StrOutputParser()

# 调用链式调用
output = chain.invoke({"input": "你是谁？20字以内回答"})

# 打印输出
print(output)

### 提示词模板工程

In [14]:
# 直接编码
from langchain_core.prompts import ChatPromptTemplate 

chat_template= ChatPromptTemplate.from_messages(
    [
        ("system","你是一个人工智能，名字叫{name}"),
        ("human","你好"),
        ("ai","我很好，谢谢"),
        ("human","{user_input}"),
    ]
) 

messages = chat_template.format_messages(name="bob",user_input="你好") 
print(messages)

[SystemMessage(content='你是一个人工智能，名字叫bob', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={})]


In [22]:
# 结构化编码
from langchain_core.prompts import ChatPromptTemplate ,HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage 

chat_template= ChatPromptTemplate.from_messages(
    [
        SystemMessage(
        content=(
            "你是一个职业秘书{name}，可以润色内容"
        )),
        HumanMessagePromptTemplate.from_template("{text}")      
    ]
)

messages = chat_template.format_messages(name="jack",text="你好，你是谁") 
print(messages)


[SystemMessage(content='你是一个职业秘书{name}，可以润色内容', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，你是谁', additional_kwargs={}, response_metadata={})]


In [40]:
# Placeholder 使用
from langchain_core.prompts import ChatPromptTemplate ,MessagesPlaceholder
from langchain_core.messages import SystemMessage ,HumanMessage

chat_template= ChatPromptTemplate.from_messages(
    [
        ("system","你是一个秘书"),
        MessagesPlaceholder("msgs")      
    ]
)

# 修复后的代码
messages = chat_template.invoke({
    "msgs": [HumanMessage(content="hi!"), HumanMessage(content="hello!")]
})
print(messages)


messages=[SystemMessage(content='你是一个秘书', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello!', additional_kwargs={}, response_metadata={})]


### 提示词最佳提示

In [2]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


examples= [
    {
        "question":"谁的寿命更长，穆罕默德·阿里还是艾伦·图灵?",
        "answer":'''
                这里需要跟进问题吗:是的。
                跟进:穆罕默德·阿里去世时多大?
                中间答案:穆罕默德·阿里去世时74岁
                跟进:艾伦·图灵去世时多大?
                中间答案:艾伦·图灵去世时41岁
                所以最终答案是:穆罕默德·阿里'''
    },
    {
        "question":"craigslist的创始人是什么时候出生的?",
        "answer":'''
                这里需要跟进问题吗:是的。
                跟进:craigslist的创始人是谁?
                中间答案:craigslist由Craig Newmark创立。
                跟进:Craig Newmark是什么时候出生的?
                中间答案:Craig Newmark于1952年12月6日出生。
                所以最终答案是:1952年12月6日'''
    },
    {
        "question":"乔治·华盛顿的祖父母中的母亲是谁?",
        "answer" : 
                '''
                这里需要跟进问题吗:是的。
                跟进:乔治1华盛顿的母亲是谁?
                中间答案:乔治·华盛顿的母亲是Mary Ball Washington。
                跟进:Mary Ball Washington的父亲是谁?
                中间答案:Mary Ball Washington的父亲是Joseph Ball。
                所以最终答案是:Joseph Ball
                '''
        },
    {
        "question":"大白鲨和皇家赌场的导演是同一个国家的吗?",
        "answer":'''
                这里需要跟进问题吗:是的。
                跟进:《大白鲨》的导演是谁?
                中间答案:《大白鲨》的导演是Steven Spielberg.
                跟进:Steven Spielberg来自哪里?
                中间答案:美国。
                跟进:《皇家赌场》的导演是谁?
                中间答案:《皇家赌场》的导演是Martin Campbell.
                跟进:Martin Campbell来自哪里?
                中间答案:新西兰。
                所以最终答案是:不是'''
    },

]

#提示词解析模板
example_prompt = PromptTemplate(input_variables=["quetion","anwser"],template="问题：{question}\\n{answer}")
 
prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="问题:{input}",
    input_variables=["input"]
)
print(prompt.format (input="《大白鲨》的导演是哪个国家的？"))
 
#只传入顶一个实例样本
print(example_prompt.format(**examples[0]))


问题：谁的寿命更长，穆罕默德·阿里还是艾伦·图灵?\n
                这里需要跟进问题吗:是的。
                跟进:穆罕默德·阿里去世时多大?
                中间答案:穆罕默德·阿里去世时74岁
                跟进:艾伦·图灵去世时多大?
                中间答案:艾伦·图灵去世时41岁
                所以最终答案是:穆罕默德·阿里

问题：craigslist的创始人是什么时候出生的?\n
                这里需要跟进问题吗:是的。
                跟进:craigslist的创始人是谁?
                中间答案:craigslist由Craig Newmark创立。
                跟进:Craig Newmark是什么时候出生的?
                中间答案:Craig Newmark于1952年12月6日出生。
                所以最终答案是:1952年12月6日

问题：乔治·华盛顿的祖父母中的母亲是谁?\n
                这里需要跟进问题吗:是的。
                跟进:乔治1华盛顿的母亲是谁?
                中间答案:乔治·华盛顿的母亲是Mary Ball Washington。
                跟进:Mary Ball Washington的父亲是谁?
                中间答案:Mary Ball Washington的父亲是Joseph Ball。
                所以最终答案是:Joseph Ball
                

问题：大白鲨和皇家赌场的导演是同一个国家的吗?\n
                这里需要跟进问题吗:是的。
                跟进:《大白鲨》的导演是谁?
                中间答案:《大白鲨》的导演是Steven Spielberg.
                跟进:Steven Spielberg来自哪里?
                中间答案:美国。
        

### 示例选择器

In [1]:
#实例选择器
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma  
from langchain_community.embeddings.ollama import OllamaEmbeddings 
 

embeddings = OllamaEmbeddings(model="bge-m3:latest") 
#语意相似性
example_selector = SemanticSimilarityExampleSelector.from_examples(#这是可供选择的示例列表。
    examples,
    #这是用于生成嵌入的嵌犬类，该嵌入用于衡量语义相似性。
    embeddings ,
    #这是用于存储嵌入和执行相似性搜索的Vectorstore类
    Chroma,
    #这是要生成的示例数
    k=1
)

#选择与输入最相似的示例。
question ="乔治·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"最相似的示例:{question}")
for example in selected_examples:
    print("\\n")
    for k,v in example.items():
        print(f"{k}:{v}")
 

C:\Users\sky92\AppData\Local\Temp\ipykernel_1104\812084880.py:57: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="bge-m3:latest")


: 

### 链式调用

In [6]:
from langchain_community.llms import Ollama
# 创建 Ollama 实例
# 确保 Ollama 服务正在运行，并且模型已经拉取到本地
llm = Ollama(
    base_url='http://127.0.0.1:11434',  # 本地 Ollama 服务地址 其实默认就是这个地址
    # model='qwen2:7b',  # 你要使用的模型名称
    model='deepseek-r1:8b',  # 你要使用的模型名称
    timeout=300
)

In [ ]:
chunks =[]
for chunk in llm.stream("天空是什么颜色的？,二十字以内！"):
    chunks.append(chunk)
    print(chunk ,end="|",flush=True)

In [ ]:
#astream chain.py  异步流式调用
# import asyncio

# #jupyter专用嵌套异步
# import nest_asyncio
# nest_asyncio.apply()

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话,快速判断，无需显示think，二十字以内！")
parser =StrOutputParser()
chain = prompt | llm | parser

async for chunk in chain.astream({"topic":"鹦鹉"}):
        print(chunk, end="",flush=True)

# async def async_stream():
#     async for chunk in chain.astream({"topic":"鹦鹉"}):
#         print(chunk, end="",flush=True)

# asyncio.run(async_stream())

### 标准json输出

In [ ]:

from langchain_core.output_parsers import JsonOutputParser
 
parser =JsonOutputParser()
chain = prompt|llm| parser
 
 

# async for chunk in chain.astream(
#     "以JSON 格式输出法国、西班牙和日本的国家及其人口列表。"
#     '使用一个带有“countries”外部键的字典，其中包含国家列表。'
#     "每个国家都应该有键`name`和`population`"
#     ): 
#         print(chunk, end="",flush=True)


for chunk in llm.stream(
    "以JSON 格式输出法国、西班牙和日本的国家及其人口列表。"
    '使用一个带有“countries”外部键的字典，其中包含国家列表。'
    "每个国家都应该有键`name`和`population`"
):
    chunks.append(chunk)
    print(chunk ,end="",flush=True)
        

In [ ]:
#示例:tools retriever.py
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.ollama import OllamaEmbeddings
from  langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(r"https://zh.wikipedia.org/wiki/%E7%8C%AB")
print(loader,"\n")
docs = loader.load()
documents =RecursiveCharacterTextSplitter(
    #chunksize 参数在 RecursivecharacterTextsplitter 中用于指定每个文档块的最大字符数。它的作
    #chunk overlap 参数用于指定每个文档块之间的重着字符数。这意味着，当文档被拆分成较小的块时，每
    # 第一个块包含字符 1 到 1088。第二个块包含字符881到 1888。
    # 第三个块包含字符 1601 到 2680。
    chunk size=1000,chunk overlap=200
    ).split documents(docs)

embeddings = OllamaEmbeddings(model="bge-m3:latest")

vector =FAISS.from_documents(documents,embeddings())
retriever =vector.as_retriever()

print(retriever.invoke("猫的特征")[0])

retriever_tool = create_retriever_tool(retriever,"wiki_search","搜索维基百科")


### AGENT调用代码

In [ ]:
# 得到提示词
from langchain import hub
#获取要使用的提示-您可以修改这个!  langchain官方的提示词仓库
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)


In [ ]:
#建立agent
from langchain.agents import create_tool_calling_agent

# 各工具集
tools=[search,retriever_tool]

agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
#开始执行agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
print(agent_executor.invoke({"input":"你好"}))
print(agent_executor.invoke({"input":"猫的特征？"}))